- Step Size: 구간(Step)에 따라 특정값으로 mapping함
→ Step Size가 클 수록 Bit의 수가 낮아짐

- 영상 파일을 압축할 때, 시간적 유사도를 바탕으로 **차분(Difference)**함

- Kernel == Window

In [ ]:
# 영상파일 읽기

import cv2
from Common.utils import print_matInfo  # 행렬 정보 출력 함수 임포트

title1, title2 = "color2gray", "color2color"
color2gray = cv2.imread("C:/Users/101/opencv/chap04/images/read_color.jpg", cv2.IMREAD_GRAYSCALE)
color2color = cv2.imread("C:/Users/101/opencv/chap04/images/read_color.jpg", cv2.IMREAD_COLOR)
if color2gray is None or color2color is None:
    raise Exception("영상 파일 읽기 에러")

print("행렬 좌표 (100, 100) 화소값")
print("%s %s" % (title1, color2gray[100, 100]))     # 한 화소값 표시
print("%s %s\n" % (title2, color2color[100, 100]))

print_matInfo(title1, color2gray)                   # 행렬 정보 출력
print_matInfo(title2, color2color)
cv2.imshow(title1, color2gray)                      # 행렬 정보 영상으로 띄우기
cv2.imshow(title2, color2color)
cv2.waitKey(0)

- ```cv.imread(file_name, flags)```: 영상을 load하여 행렬로 변환
    - flags: load한 영상을 행렬로 변환할 때 칼러 타입을 결정
        - 0: cv2.IMREAD_GRAYSCALE, GrayScale 형태로 변환
        - -1: cv2.IMREAD_UNCHANGED, 영상 그대로 변환(alpha 채널 포함)
        - 1: cv2.IMREAD_COLOR, 칼러 영상으로 변환하여 반환

In [ ]:
import cv2
from Common.utils import print_matInfo

title1, title2 = "16bit unchanged", "32bit unchanged"  # 윈도우 이름
color2unchanged1 = cv2.imread("images/read_16.tif", cv2.IMREAD_UNCHANGED)
color2unchanged2 = cv2.imread("images/read_32.tif", cv2.IMREAD_UNCHANGED)
if color2unchanged1 is None or color2unchanged2 is None:
    raise Exception("영상파일 읽기 에러")

print("16/32비트 영상 행렬 좌표 (10, 10) 화소값")
print(title1, "원소 자료형 ",  type(color2unchanged1[10][10][0]))   # 원소 좌료형
print(color2unchanged1[10][10][0])
print(color2unchanged1[10][10][1])
print(color2unchanged1[10][10][2])
print(title1, "화소값(3원소) ", color2unchanged1[10, 10] )           # 행렬 내 한 화소 값 표시
print(title2, "원소 자료형 ",  type(color2unchanged2[10][10][0]))
print(title2, "화소값(3원소) ", color2unchanged2[10, 10] )
print()

print_matInfo(title1, color2unchanged1)         # 행렬 정보 출력
print_matInfo(title2, color2unchanged2)
cv2.imshow(title1, color2unchanged1)
cv2.imshow(title2, (color2unchanged2*255).astype('uint8')) # 8bit로 변환
cv2.waitKey(0)

- ```cv2.imshow(title2, (color2unchanged2*255).astype('uint8'))```
    - 255로 곱하기 Upscaling

# 카메라에서 프레임 읽기

In [ ]:
import cv2

def put_string(frame, text, pt, value, color=(120, 200, 90)): # 문자열 출력 함수 - 그림자 효과
    text += str(value)
    shade = (pt[0] + 2, pt[1] + 2)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, text, shade, font, 0.7, (0, 0, 0), 2)  # 그림자 효과
    cv2.putText(frame, text, pt, font, 0.7, (120, 200, 90), 2)  # 글자 적기

capture = cv2.VideoCapture(0)  # 0번 카메라 연결
if capture.isOpened() == False:
    raise Exception("카메라 연결 안됨")

# 카메라 속성 획득 및 출력
print("너비 %d" % capture.get(cv2.CAP_PROP_FRAME_WIDTH))
print("높이 %d" % capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("노출 %d" % capture.get(cv2.CAP_PROP_EXPOSURE))
print("밝기 %d" % capture.get(cv2.CAP_PROP_BRIGHTNESS))

while True:  # 무한 반복
    ret, frame = capture.read()  # 카메라 영상 받기
    if not ret: break
    if cv2.waitKey(30) >= 0: break # 종료조건: Event key 32는 스페이스바 키를 나타냄, https://lifeones.tistory.com/104

    exposure = capture.get(cv2.CAP_PROP_EXPOSURE)
    put_string(frame, "EXPOS: ", (10, 40), exposure)
    title = "View Frame from Camera"
    cv2.imshow(title, frame)  # 윈도우에 영상 띄우기
capture.release()

- ```cv2.VideoCapture()```: 생성자
- ```cv2.VideoCapture.release()```: 소멸자
- ```cv2.VideoCapture.get(propid)```: 지정된 속성값을 반환
    - propid: property id (변경하려는 카메라 설정)
- ```cv2.waitKey(30) >= 0``` # 종료조건: Event key 32는 스페이스바 키를 나타냄, https://lifeones.tistory.com/104

- ```cv2.split(다채널)``` -> 단일채널
- ```cv.merge(단일채널)``` -> 다채널

# OpenCV 기본연산

- OpenCV는 기본적으로 Saturation 연산으로 화소값 연산을 함
    - Saturation 연산: 한계값을 정하고, 그 값을 벗어나는 경우에는 특정값으로 계산하는 방식
    - ex) image에서 0 이하는 모두 0, 255 이상은 모두 255로 표현

- **(중요)** 논리(비트) 연산 함수
    - OpenCV에서 영상의 논리 연산(logical operation)이란 픽셀 값을 2진수로 표현하여 각 비트 단위 논리 연산을 수행하는 것을 의미한다.

    - OpenCV에서는 논리합(OR), 논리곱(AND), 부정(NOT), 배타적 논리합(XOR) 연산을 지원한다.

        - 논리합(OR) : 2개의 입력 비트 중 1개라도 1이면 결과가 1이 되는 연산

        - 논리곱(AND) : 2개의 입력 비트 중 모두 1인 경우만 결과가 1이 되는 연산

        - 부정(NOT) : 입력 비트가 0이면 결과가 1, 1이면 결과가 0이 되는 연산

        - 배타적 논리합(XOR) : 2개의 입력 비트 중 1개만 1인 경우 결과가 1이 되는 연산